In [ ]:
#import sys
#print(sys.path)
#sys.path.insert(0,'/afs/desy.de/user/y/yian/.local/bin')
#print(sys.path)

In [ ]:
import tensorflow
tensorflow.__file__

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

In [ ]:
import uproot
import awkward as ak
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm

In [ ]:
basedir = '/nfs/dust/cms/user/yian/ajj/'
year="18"
root_sig = glob.glob(basedir+"*EW*"+year+".root")
root_bkg = glob.glob(basedir+"*GJets*"+year+".root")

In [ ]:
print(root_bkg,root_sig)

In [ ]:
branch = uproot.open(root_sig[1]+":Events").keys()
necessary_columns=[]
unnecessary_columns=['vjj_jets','vjj_mus','vjj_eles','vjj_photons','vjj_loosePhotons']
for i in branch:
    if ('vjj_' in i and 'gen' not in i and i not in unnecessary_columns) or ('eff' in i) or ('scalef' in i):        
        necessary_columns.append(i)
print('select necessary branches')

In [ ]:
signal = uproot.lazy(root_sig[1]+':Events')[necessary_columns]
sig=pd.DataFrame(signal[necessary_columns].to_numpy()).query("vjj_isGood==1 & vjj_jj_m>500 & vjj_v_pt>200 & vjj_lead_pt>50 & vjj_sublead_pt>50 & vjj_trig>0")
background=uproot.lazy(root_bkg[0]+':Events')[necessary_columns]
bkg=pd.DataFrame(background[necessary_columns].to_numpy()).query("vjj_isGood==1 & vjj_jj_m>500 & vjj_v_pt>200 & vjj_lead_pt>50 & vjj_sublead_pt>50 & vjj_trig>0")
sig['isSignal']=1
bkg['isSignal']=0

In [ ]:
sig['vjj_dphi']=abs(sig['vjj_v_phi']-sig['vjj_jj_phi'])
bkg['vjj_dphi']=abs(bkg['vjj_v_phi']-bkg['vjj_jj_phi'])
bkg['vjj_zepp']=abs(bkg['vjj_v_eta']-(bkg['vjj_lead_eta']+bkg['vjj_sublead_eta'])/2)
sig['vjj_zepp']=abs(sig['vjj_v_eta']-(sig['vjj_lead_eta']+sig['vjj_sublead_eta'])/2)               

In [ ]:
sig.loc[sig.query('vjj_dphi>3.14').index,'vjj_dphi']=2*3.14-sig.query('vjj_dphi>3.14')['vjj_dphi']
bkg.loc[bkg.query('vjj_dphi>3.14').index,'vjj_dphi']=2*3.14-bkg.query('vjj_dphi>3.14')['vjj_dphi']

In [ ]:
sig['isBkg']=0
bkg['isBkg']=1

In [ ]:
df = pd.concat([sig,bkg],ignore_index=True)
print('combine signal and background') 

In [ ]:
features=['vjj_jj_m','vjj_jj_dphi','vjj_jj_pt','vjj_jj_deta','vjj_v_ystar','vjj_vjj_aplanarity','vjj_vjj_C','vjj_vjj_dphi','vjj_vjj_circularity','vjj_sublead_pt','vjj_lead_qgl','vjj_sublead_qgl']
features.append('vjj_lead_pt')
features.append('vjj_vjj_deta')
features.append('vjj_vjj_isotropy')
features.append('vjj_vjj_eta')
features.append('vjj_zepp')
features.append('vjj_dphi')

In [ ]:
X=df[features]
Y=df[['isSignal','isBkg']]

In [ ]:
print(X.shape,Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

In [ ]:
df.loc[X_train.index,'scalef']

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

model = Sequential()

# model.add: to add a layer to our neural network
model.add(Dense(18, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='softmax'))
# show a summary
model.summary()

# Specify the loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=500)#sample_weight=df.loc[X_train.index,'scalef']
                    

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
import boost_histogram as bh
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep as hep

In [ ]:
np.where(np.array(y_test)[:, 0] == True)[0]

In [ ]:
y_train

In [ ]:
y_pred = model.predict(X_train)
y_pred_sig_score = y_pred[:, 0] # score predicted for each event to be a "signal event"

sig_index = np.where(np.array(y_train)[:, 0] == True)[0]
bkg_index = np.where(np.array(y_train)[:, 0] == False)[0]
f, ax = plt.subplots(figsize=(8, 8))
for idx, lab in zip([sig_index, bkg_index], ['EW signal', 'QCD background']):
    hist = bh.Histogram(bh.axis.Regular(50, 0., 1.), storage=bh.storage.Weight())
    hist.fill(y_pred_sig_score[idx])
    content, yerr = hist.view().value, np.sqrt(hist.view().variance)
    # plot the histogram
    hep.histplot(content, bins=hist.axes[0].edges, yerr=yerr, label=lab)
ax.set_xlabel('NN score (signal-like)', ha='right', x=1.0)
ax.set_ylabel('Events / bins', ha='right', y=1.0)
ax.set_xlim(0, 1)
ax.set_yscale('symlog')
ax.legend()

In [ ]:
y_pred = model.predict(X_test)
y_pred_sig_score = y_pred[:, 0] # score predicted for each event to be a "signal event"

sig_index = np.where(np.array(y_test)[:, 0] == True)[0]
bkg_index = np.where(np.array(y_test)[:, 0] == False)[0]
f, ax = plt.subplots(figsize=(8, 8))
for idx, lab in zip([sig_index, bkg_index], ['EW signal', 'QCD background']):
    hist = bh.Histogram(bh.axis.Regular(50, 0., 1.), storage=bh.storage.Weight())
    hist.fill(y_pred_sig_score[idx])
    content, yerr = hist.view().value, np.sqrt(hist.view().variance)
    # plot the histogram
    hep.histplot(content, bins=hist.axes[0].edges, yerr=yerr, label=lab)
ax.set_xlabel('NN score (signal-like)', ha='right', x=1.0)
ax.set_ylabel('Events / bins', ha='right', y=1.0)
ax.set_xlim(0, 1)
ax.legend()